In [1]:
from google.colab import files
import pickle as pkl
import pandas as pd
import re


uploads=files.upload()    # upload the dataset

Saving Travel_dataset_final_V3.csv to Travel_dataset_final_V3.csv


In [2]:
import re

repeated = [] 
invalid = []
df = pd.read_csv('Travel_dataset_final_V3.csv')

# removing the rows where the place names are same
for i in range(len(df)-1):
  for j in range(i+1,len(df)):
    if df['title'][j].lower() == df['title'][i].lower():
      repeated.append(j)
df.drop(repeated,axis=0,inplace=True)

# editing the place titles to replace non alphanumeric characters
for i in range(len(df)):
  df['title'][i] = re.sub("[^a-zA-Z0-9\s]",'', df['title'][i])
  if df['title'][i] == ' ' or df['title'][i]=='':
    invalid.append(i)
df.drop(invalid,axis=0,inplace=True)
df

<ipython-input-2-67aebc2f849d>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title'][i] = re.sub("[^a-zA-Z0-9\s]",'', df['title'][i])


KeyError: ignored

In [23]:
df.to_csv('Travel_dataset_final.csv', index=False)
files.download('Travel_dataset_final.csv')  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
import pickle as pkl
import pandas as pd
import re


def preprocess_df(df):
  for col in df.columns:
    if len(re.findall("imageUrls",col))==0 and col != 'place_id' and col !='postalCode' and col !='location_lat' and col !='location_lng' and col !='final_score':
      df[col]=[str(txt).lower() for txt in df[col]]
  return df

df = pd.read_csv('Travel_dataset_final_V3.csv')
df = preprocess_df(df)
df

place_id                     title       categoryName  \
0         2848       sri harmandir sahib         pilgrimage   
1         3368   gateway of india mumbai  historical_places   
2         1147             mysore palace  historical_places   
3          112                india gate  historical_places   
4         1847                 taj mahal  historical_places   
...        ...                       ...                ...   
3695       689  view point assamnagaland          adventure   
3696       663          tizu picnic spot          adventure   
3697       662                  koridang          adventure   
3698       652     shuwatho fishery pond          adventure   
3699       988            chhimveng kawn          adventure   

                                                address      neighborhood  \
0     golden temple rd, atta mandi, katra ahluwalia,...  golden temple rd   
1     apollo bandar, colaba, mumbai, maharashtra 400...                 0   
2     sayyaji rao rd, agrahara, chamrajpura, mysuru,...    sayyaji rao rd   
3     kartavya path, india gate, new delhi, delhi 11...     kartavya path   
4     dharmapuri, forest colony, tajganj, agra, utta...                 0   
...                                                 ...               ...   
3695            rrmc+2w, borjan, nagaland 798622, india           rrmc+2w   
3696                     prhx+gjf, moya, kiphire, india    prhx+gjf, moya   
3697          7fvm+5gp, aolijen, nagaland 798601, india          7fvm+5gp   
3698             jj7h+66c, phek, nagaland 797114, india          jj7h+66c   
3699           cxv2+x9w, khawbel, mizoram 796161, india          cxv2+x9w   

                street       city  postalCode          state  location_lat  \
0     golden temple rd   amritsar      143006         punjab     31.619980   
1                    0     mumbai           0    maharashtra     18.921984   
2       sayyaji rao rd     mysuru      570001      karnataka     12.305163   
3        kartavya path  new delhi      110001          delhi     28.612912   
4                    0       agra      282001  uttar pradesh     27.175145   
...                ...        ...         ...            ...           ...   
3695           rrmc+2w     borjan           0       nagaland     26.832550   
3696    prhx+gjf, moya    kiphire           0       nagaland     25.728818   
3697          7fvm+5gp    aolijen      798601       nagaland     26.293500   
3698          jj7h+66c       phek           0       nagaland     25.613074   
3699          cxv2+x9w    khawbel      796161        mizoram     23.444986   

      ...         categories_2      categories_3  categories_4 categories_5  \
0     ...            gurudwara                 0             0            0   
1     ...             monument  historical place             0            0   
2     ...                    0                 0             0            0   
3     ...  historical landmark          monument  war memorial            0   
4     ...             monument                 0             0            0   
...   ...                  ...               ...           ...          ...   
3695  ...                    0                 0             0            0   
3696  ...                    0                 0             0            0   
3697  ...                    0                 0             0            0   
3698  ...                    0                 0             0            0   
3699  ...                    0                 0             0            0   

     categories_6                                     reviews_0_text  \
0               0  for the sikhs, this location is sacred. you mu...   
1               0  the gateway of india is undoubtedly one of the...   
2               0  mysore palace is one of the most beautiful arc...   
3               0  must visit place ,there are so many parks and ...   
4               0  this should be six star, one extra for the fee...   
...   

In [30]:
class category_index:# class for the inverted index storing tourist attractions' indices based on category
  def __init__(self,df):
    self.dataframe=df
    self.index_dict=dict()
    self.cat_list=sorted([st.lower() for st in df['categoryName'].unique()])

  def get_cat_index(self,cat):
    cat_df=self.dataframe[self.dataframe['categoryName']==cat]# getting the indices based on a given category
    return list(cat_df.index.values)

  def make_index(self):
    for cat in self.cat_list:
      self.index_dict[cat.lower()]=self.get_cat_index(cat)# making index
      
  def get_cat_list(self):
    return self.cat_list


In [31]:
class state_index:# class for the inverted index storing tourist attractions' indices based on state
  def __init__(self,df):
    self.dataframe=df
    self.index_dict=dict()
    self.state_list=sorted([st.lower() for st in df['state'].unique()])

  def get_st_index(self,st):# getting the indices based on a given state
    st_df=self.dataframe[self.dataframe['state']==st]
    return list(st_df.index.values)

  def make_index(self):# making index
    for st in self.state_list:
      self.index_dict[st.lower()]=self.get_st_index(st)

  def get_state_list(self):
    return self.state_list


In [32]:
class city_index:# class for the inverted index storing tourist attractions' indices based on city
  def __init__(self,df):
    self.dataframe=df
    self.index_dict=dict()
    self.city_list=sorted([st.lower() for st in df['city'].unique()])

  def get_city_index(self,ct):# getting the indices based on a given city
    ct_df=self.dataframe[self.dataframe['city']==ct]
    return list(ct_df.index.values)

  def make_index(self):# making index
    for ct in self.city_list:
      self.index_dict[ct.lower()]=self.get_city_index(ct)

  def get_ct_list(self):
    return self.city_list


In [33]:
class sub_cat_index: #class for the inverted index storing tourist attractions' indices based on a subcategory
    def __init__(self,df):
      self.dataframe=df
      self.index_dict=dict()
      self.sub_cat_list=sorted([s.lower() for s in df['SubcategoryName'].unique()])

    def get_sub_cat_index(self,sc):
      sub_cat_df=self.dataframe[self.dataframe['SubcategoryName']==sc]
      return list(sub_cat_df.index.values)

    def make_index(self): 
      for sc in self.sub_cat_list:
          self.index_dict[sc.lower()]=self.get_sub_cat_index(sc)

    def get_sub_cat_list(self):
      return self.sub_cat_list

In [34]:
class all_places:
  def __init__(self,df):
    self.dataframe = df
    self.place_dict = dict()
    self.place_list = self.make_place_list()

  def make_place_list(self):
    place_list = []
    for col in self.dataframe.columns:
      if col == 'title' or col == 'peopleAlsoSearch_0_title' or col == 'peopleAlsoSearch_1_title' or col == 'peopleAlsoSearch_2_title' or col == 'peopleAlsoSearch_3_title' or col == 'peopleAlsoSearch_4_title':
        places = [[p,col] for p in self.dataframe[col] if p != '0' and p != 'nan' and p != ' ' and p !='']
        place_list.extend(places)
    return place_list

  def get_place_index(self,key,col):
    place_df=self.dataframe[self.dataframe[col]==key]
    return list(place_df.index.values)

  def make_index(self):
    for pl in self.place_list:
      self.place_dict[pl[0]] = []
    for pl in self.place_list:
      self.place_dict[pl[0]].extend(self.get_place_index(pl[0],pl[1]))
    for pl in self.place_dict.keys():
      if len(self.place_dict[pl]) != len(list(set(self.place_dict[pl]))):
        index = self.place_dict[pl]
        for i in range(0,len(index)-1):
          if index[i] != -1:
            for j in range(i+1,len(index)):
              if index[i] == index[j]:
                index[j] = -1
        self.place_dict[pl] = [i for i in index if i!=-1]


  def get_place_list(self):
    return [item[0] for item in self.place_list]


In [35]:
from math import sin,cos,asin,radians,sqrt
import numpy as np
class distance_index:
  
  def __init__(self,df):
    self.df=df
    self.index=dict()
    self.near_by_place=dict()
    self.THRESHOLD=50

  def get_distance(self,lat1,lat2,long1,long2):
    lon1 = radians(long1)
    lon2 = radians(long2)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * asin(sqrt(a))    
    # Radius of earth in kilometers. Use 3956 for miles
    r = 6371      
    # calculate the result
    return(c * r)
    
  def make_index_keys(self):
    places=self.df['title']
    indx=self.df.index
    for i in range(len(places)-1):
      for j in range(1,len(places)):
        lat1=self.df.at[i,'location_lat']
        lat2=self.df.at[j,'location_lat']
        long1=self.df.at[i,'location_lng']
        long2=self.df.at[j,'location_lng']
        dist=self.get_distance(lat1,lat2,long1,long2)
        pl1=places[i]
        pl2=places[j]
        tup=tuple([pl1,pl2])
        self.index[tup]=dist

  def make_nearby_places(self):
    for pl in self.df['title']:
      self.near_by_place[pl]=list()
      for key in self.index.keys():
        if pl==key[0] and self.index[key]<=self.THRESHOLD:
          self.near_by_place[pl].append(key[1])
        elif pl==key[1] and self.index[key]<=self.THRESHOLD:
          self.near_by_place[pl].append(key[0])

  def make_nearby_places2(self):
    keys = list(self.index.keys())
    values = list(self.index.values())
    sorted_value_index = np.argsort(values)
    sorted_dict = {keys[i]: values[i] for i in sorted_value_index}
    for pl in self.df['title']:
      self.near_by_place[pl]=list()
    for key in sorted_dict:
      if sorted_dict[key]<=self.THRESHOLD:
        self.near_by_place[key[0]].append(key[1])
        self.near_by_place[key[1]].append(key[0])
      else:
        break
    for key in self.near_by_place:
      self.near_by_place[key]=list(set(self.near_by_place[key]))
    
    

In [27]:
di=distance_index(df)
di.make_index_keys()
di.make_nearby_places2()

KeyboardInterrupt: ignored

In [36]:
import pandas as pd
from google.colab import files
import pickle as pkl

cai = category_index(df)
cai.make_index()
fobj=open('cat_index.pkl','wb')
pkl.dump(cai,fobj)
fobj.close()

scai = sub_cat_index(df)
scai.make_index()
fobj1=open('subcat_index.pkl','wb')
pkl.dump(scai,fobj1)
fobj1.close()

si = state_index(df)
si.make_index()
fobj2=open('state_index.pkl','wb')
pkl.dump(si,fobj2)
fobj2.close()

ci = city_index(df)
ci.make_index()
fobj3=open('city_index.pkl','wb')
pkl.dump(ci,fobj3)
fobj3.close()


pi = all_places(df)
pi.make_index()
fobj4=open('place_name_index.pkl','wb')
pkl.dump(pi,fobj4)
fobj4.close()

di=distance_index(df)
di.make_index_keys()
di.make_nearby_places2()
foj=open("near_by_places.pkl",'wb')
pkl.dump(di,foj)
foj.close()


files.download('cat_index.pkl')   
files.download('subcat_index.pkl')     
files.download('state_index.pkl')     
files.download('city_index.pkl')  
files.download('place_name_index.pkl')  
files.download('near_by_places.pkl')  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
from google.colab import files
uploads=files.upload()

Saving state_acr_new.txt to state_acr_new.txt


In [38]:
st_syn_obj=open('/content/state_acr_new.txt','r')
st_syn_text=st_syn_obj.read()
st_syn_dict=dict()
lines_st=st_syn_text.split(";")
for ln in lines_st:# creating the dictionary to store the synonyms for the state
  ln_spl=ln.split(":")
  wd1=ln_spl[1].replace('\n','')
  wd1=wd1.strip()
  st_syn_dict[str(wd1).lower()]=str(ln_spl[0]).replace("\n","").lower()
  st_syn_dict[str(ln_spl[0]).replace("\n","").lower()]=str(ln_spl[0]).replace("\n","").lower()

In [39]:
print(st_syn_dict)

{'ap': 'andhra pradesh', 'andhra pradesh': 'andhra pradesh', 'ar': 'arunachal pradesh', 'arunachal pradesh': 'arunachal pradesh', 'as': 'assam', 'assam': 'assam', 'br': 'bihar', 'bihar': 'bihar', 'cg': 'chhattisgarh', 'chhattisgarh': 'chhattisgarh', 'ga': 'goa', 'goa': 'goa', 'gj': 'gujarat', 'gujarat': 'gujarat', 'hr': 'haryana', 'haryana': 'haryana', 'jammu and kashmir': 'himachal pradesh', 'himachal pradesh': 'himachal pradesh', 'jh': 'jharkhand', 'jharkhand': 'jharkhand', 'ka': 'karnataka', 'karnataka': 'karnataka', 'kl': 'kerala', 'kerala': 'kerala', 'mp': 'madhya pradesh', 'madhya pradesh': 'madhya pradesh', 'mh': 'maharashtra', 'maharashtra': 'maharashtra', 'mn': 'manipur', 'manipur': 'manipur', 'ml': 'meghalaya', 'meghalaya': 'meghalaya', 'mz': 'mizoram', 'mizoram': 'mizoram', 'nl': 'nagaland', 'nagaland': 'nagaland', 'or': 'odisha', 'odisha': 'odisha', 'pb': 'punjab', 'punjab': 'punjab', 'rj': 'rajasthan', 'rajasthan': 'rajasthan', 'sk': 'sikkim', 'sikkim': 'sikkim', 'tn': 'ta

In [40]:
import pickle as pkl   # synonym pickling
fobj1=open("synonym_st_dict.pkl",'wb')
pkl.dump(st_syn_dict,fobj1)
fobj1.close()
files.download('synonym_st_dict.pkl')     

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
class MainIndex:# creating the cross index, it will store all the indices for each of category, subcategory, city, state. Also it will store
  def __init__(self):# index lists which satisfies intersection of a city/state and category
    self.cat_i=None
    self.st_i=None
    self.city_i=None
    self.cross_index=dict()
    self.cross_list=list()

  def get_indices(self):# loading the index objects
    fobj1=open('cat_index.pkl','rb')
    self.cat_i=pkl.load(fobj1)
    fobj1.close()
    fobj2=open('state_index.pkl','rb')
    self.st_i=pkl.load(fobj2)
    fobj2.close()
    fobj3=open('city_index.pkl','rb')
    self.city_i=pkl.load(fobj3)
    fobj3.close()
    fobj4=open('near_by_places.pkl','rb')
    self.nb_places_i=pkl.load(fobj4)
    fobj4.close()


  def merger(self,cat,st='',ct=''):# doing the intersection of category and state/city
    if len(ct)>0:
      l1=self.cat_i.get_cat_index(cat)
      l2=self.ct_i.get_ct_index(ct)
      fin_index=set(l1).intersection(set(l2))
      return list(fin_index)
    elif len(st)>0:
      l1=self.cross_index[cat]
      l2=self.cross_index[st]
      fin_index=set(l1).intersection(set(l2))
      return list(fin_index)
    else:
      return []

  def create_cross_index(self):# creating the index , storing category and state combination as c or (category,city) for key in dictionary
    self.get_indices()
    for cat in self.cat_i.get_cat_list():
        self.cross_index[cat]=list()
        self.cross_index[cat].extend(self.cat_i.get_cat_index(cat))
    for st in self.st_i.get_state_list():
      self.cross_index[st]=list()
      self.cross_index[st].extend(self.st_i.get_st_index(st))
    for city in self.city_i.get_ct_list():
        if city in self.st_i.get_state_list():
              self.cross_index[city].extend(self.city_i.get_city_index(city))
        else:
              self.cross_index[city]=list()
              self.cross_index[city].extend(self.city_i.get_city_index(city))
    for cat in self.cat_i.get_cat_list():# making intersection of indices, (category,state)
      for st in self.st_i.get_state_list():
        tup=tuple([cat,st])
        self.cross_index[tup]=self.merger(cat,st)
    for cat in self.cat_i.get_cat_list():# making intersection of indices, (category,city)
      for ct in self.city_i.get_ct_list():
        tup=tuple([cat,ct])
        self.cross_index[tup]=self.merger(cat,ct)

  def index_lookup(self,cat='',pl=''):# facilitating look up of index
    if len(cat)>0:
      if len(pl)>0:
        tup=tuple([cat,pl])
        return self.cross_index[tup]
      else:
        return self.cross_index[cat]
    elif len(pl)>0:
      return self.cross_index[pl]
    else:
      return []


In [42]:
#uploads=files.upload()            # upload cat_index, state_index, city_index
mi = MainIndex()
mi.create_cross_index()
fobj=open('cross_index.pkl','wb')
pkl.dump(mi,fobj)
fobj.close()
files.download('cross_index.pkl')     

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [43]:
!pip install pattern
!pip install textdistance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 23.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.1/88.1 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 68.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.4/348.4 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pattern: filename=Pattern-3.6-py3-none-any.whl size=22332718 sha256=0547aa110bb8b786fd11516383d0c6e276d5a8975ebb96276e1858530bfc2a6b
  Stored in directory: 

In [44]:
import re
import pickle as pkl  
from google.colab import files
from bisect import bisect_left
import nltk
import pattern
from pattern.en import singularize
from collections import OrderedDict
import textdistance
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words=list(stopwords.words("english"))
MAX_NGRAMS=3

#uploads=files.upload()        # upload the 7 indices and 1 synonym mapping dicts                    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [45]:
class Query_Processing:# class for processing the query
  def __init__(self):
    self.cat_index=None
    self.state_index=None
    self.city_index=None
    self.cross_index = None
    self.res_df = []
    self.st_syn_dict = None
    self.sub_cat_index = None
    self.nb_places_i = None
    self.place_index = None
    self.bigram = []
  
  def get_indices(self):
    fobj1=open('cat_index.pkl','rb')
    self.cat_index=pkl.load(fobj1)
    fobj1.close()
    fobj2=open('state_index.pkl','rb')
    self.state_index=pkl.load(fobj2)
    fobj2.close()
    fobj3=open('city_index.pkl','rb')
    self.city_index=pkl.load(fobj3)
    fobj3.close()
    fobj5=open('synonym_st_dict.pkl','rb')
    self.st_syn_dict=pkl.load(fobj5)
    fobj5.close()
    fobj6=open('cross_index.pkl','rb')
    self.cross_index=pkl.load(fobj6)
    fobj6.close()
    fobj7=open('near_by_places.pkl','rb')
    self.nb_places_i=pkl.load(fobj7)
    fobj7.close()
    fobj8=open('subcat_index.pkl','rb')
    self.sub_cat_index=pkl.load(fobj8)
    fobj8.close()
    fobj9=open('place_name_index.pkl','rb')
    self.place_index=pkl.load(fobj9)
    fobj9.close()


  
  def preprocess_query(self,query):# query preprocesser
    query = query.lower()                                        # converting the texts of query string into lowercase
    words = re.sub('[^\w\s]',' ',query)                        # removing non-alphanumeric and non-space characters from the query string
    tokens = nltk.word_tokenize(words)                           # tokenizing the modified query string
    words_list = [t for t in tokens if t not in stop_words]      # removing stopwords from the tokens obtained in last step
    for k in self.st_syn_dict.keys():
        for i in range(len(words_list)):
            if words_list[i] == k:
                words_list[i] = self.st_syn_dict[k]
    words_list = [t for t in words_list if t!=' ' and t!='']               # removing all blank-space and empty tokens
    if(len(words_list)>10):                                      
      words_list = words_list[:10]
    words_list_new=[wd for wd in words_list]
    for i in range(2,MAX_NGRAMS+1):
      ngram_list=nltk.ngrams(words_list,i)
      for ng in ngram_list:
        ng_st=' '.join(ng)
        words_list_new.append(ng_st)
    return words_list_new



  def make_ngrams(self,words_list,n,cat_list,index_dict):
    categories=list()
    if n >=2:
        for i in range(len(words_list)-(n-1)):#finding categories
            cat = ' '.join(words_list[i:i+n])
            if cat in cat_list:
              categories.append([cat,index_dict[words_list[i]]])
        return categories
    else:
      return list()


  def make_bigrams(self,words_list):
    index_dict = {word:i for i,word in enumerate(words_list)}           # storing position of words in the query
    categories, places, bigram, sub_categories = [],[],[],[]
    cat_list, city_list, state_list,sub_cat_list = self.cat_index.get_cat_list(), self.city_index.get_ct_list(), self.state_index.get_state_list(),self.sub_cat_index.get_sub_cat_list()
    for w in words_list:
      if self.binary_search(cat_list,w):
        categories.append([w,index_dict[w]])
      else:
        w_s = singularize(w)
        if self.binary_search(cat_list,w_s):
          categories.append([w_s,index_dict[w]])
    for w in words_list:
      if self.binary_search(sub_cat_list,w):
        sub_categories.append([w,index_dict[w]])
      else:
        w_s = singularize(w)
        if self.binary_search(sub_cat_list,w_s):
          categories.append([w_s,index_dict[w]])
    for i in range(2,MAX_NGRAMS+1):
      cat_new=self.make_ngrams(words_list,i,cat_list,index_dict)
      categories.extend(cat_new)
    for i in range(2,MAX_NGRAMS+1):
      sub_cat_new=self.make_ngrams(words_list,i,sub_cat_list,index_dict)
      sub_categories.extend(sub_cat_new)
    for w in words_list:          # finding the words in query which correspond to state or city
      if self.binary_search(city_list,w):
        places.append([w,index_dict[w]])
      elif self.binary_search(state_list,w):
        places.append([w,index_dict[w]])
    for i in range(len(words_list)-1):        # storing phrases which correspond to city/state
        plc = ' '.join(words_list[i:i+2])
        if self.binary_search(city_list,plc):
          places.append([plc,index_dict[words_list[i]]])
        elif self.binary_search(state_list,plc):
          places.append([plc,index_dict[words_list[i]]])
    categories.sort(key = lambda x: x[1])
    places.sort(key = lambda x: x[1])
    if len(categories) == len(places):
      for i in range(len(places)):
        self.bigram.append(tuple([categories[i][0],places[i][0]]))# making bigrams on categories places, just based on their locations
    elif len(categories)>0 and len(places)>0:
        for i in range(len(categories)):# making all possible combinations of places and categories
            for j in range(len(places)):
              tup = tuple([categories[i][0],places[j][0]])
              if tup in self.cross_index.cross_index:
                self.bigram.append(tup)
    elif len(categories)>0:# only categories
        for i in range(len(categories)):
            self.bigram.append(categories[i][0])
    elif len(places)>0:
        for i in range(len(places)):# only places
            self.bigram.append(places[i][0])



  def predict_place(self,input):
    input = input.lower()
    out = []
    all_words = self.place_index.get_place_list()
    for wd in all_words:
      sim = 1-(textdistance.Jaccard(qval=2).distance(input,wd))
      if sim >= 0.5:
        out.append([wd,sim])
    out.sort(key = lambda x: x[1], reverse = True)
    if len(out) == 0:
      return None, None
    return [out[:5],out[0][0]] if len(out) >= 5 else [out,out[0][0]]

      
  def check_place(self,query):
    res_df = []
    top_place = None
    if query in self.place_index.place_dict:
      res_df = df.loc[self.place_index.place_dict[query]]
      top_place = query
    else:
      places, top_place = self.predict_place(query)
      if places != None:
        place_n = [pl[0] for pl in places]
        top_place = place_n[0]
        res_df = df.loc[self.place_index.place_dict[place_n[0]]]
        for i in range(1,len(places)):
          curr = df.loc[self.place_index.place_dict[place_n[i]]]
          res_df = pd.concat([curr,res_df])
    return res_df, top_place


  def get_filtered_res(self,data,qwords):
    indexes = list(data.index.values)
    sim_score = [[id,0] for id in indexes]
    for i in range(len(indexes)):
      ft = str(data.loc[indexes[i]]['feature_set']).split()
      sim_score[i][1] = len(set(ft).intersection(set(qwords)))/len(qwords)
    sim_score.sort(key = lambda x: x[1], reverse = True)
    if len(data) <= 100:
      res_id = [ele[0] for ele in sim_score if (ele[1]*len(qwords))>=1]
    else:
      res_id = [ele[0] for ele in sim_score[:100] if (ele[1]*len(qwords))>=1]
    res_df = data.loc[res_id]
    return res_df


  def binary_search(self,a, x):
    i = bisect_left(a, x)
    if i != len(a) and a[i] == x:
        return i
    else:
        return 0


  def find_nearby_places(self,place):
    return self.nb_places_i.near_by_place[place]




  def query_processor(self,query):# query processor
    self.get_indices()
    self.res_df, top = self.check_place(query.lower())
    if len(self.res_df) > 0:
      if top != None:
        print("-----near by places to -----",top)
        print(self.find_nearby_places(top))
        print("-----------------------------------")
      return self.res_df
    words_list = self.preprocess_query(query)
    self.make_bigrams(words_list)
    res_indices = set()
    df = self.cat_index.dataframe
    for bg in self.bigram:# find the list of indices based on the bigrams created
      if type(bg)==tuple():
          res = self.cross_index.index_lookup(bg[0],bg[1])
      else:
          res = self.cross_index.index_lookup(bg)
      res_indices = res_indices.union(res)
    if len(res_indices) > 0 :
      self.res_df = df.iloc[list(res_indices)]
    else:
      self.res_df = df
    self.res_df = self.get_filtered_res(self.res_df,words_list)
    return self.res_df

  

In [47]:
q = Query_Processing()# creating an object
query = input("Enter a search query : ")
res_df = q.query_processor(query)
res_df

Enter a search query : taj mahal
-----near by places to ----- taj mahal
['agra fort', 'great gate taj mahal', 'kanch mahal', 'tomb of akbar the great', 'holy tomb of hazrat salim chishti', 'shah jahani mahal', 'birbals palace', 'shahi burj', 'taj mahal', 'taj yamuna view point', 'diwaniam', 'soor sarovar bird sanctuary', 'panch mahal', 'sikandra fort', 'taj nature walk', 'anguri bagh', 'fatehpur sikri fort']
-----------------------------------


place_id                    title       categoryName  \
4         1847                taj mahal  historical_places   
1756      1776    taj yamuna view point          adventure   
516       1843  tomb of akbar the great  historical_places   
1056      1774     great gate taj mahal          adventure   
15        1850                agra fort  historical_places   
1703      1820              kanch mahal          adventure   
2913      1795        shah jahani mahal             museum   

                                                address  \
4     dharmapuri, forest colony, tajganj, agra, utta...   
1756  52gv+ch2, dharmapuri, forest colony, tajganj, ...   
516   tomb of akbar the great area, sikandra, agra, ...   
1056  great gate, 282001, dharmapuri, forest colony,...   
15    agra fort, rakabganj, agra, uttar pradesh 2820...   
1703  sikandra - bhagwan talkies rd, tomb of akbar t...   
2913  52hf+69m, agra fort, rakabganj, agra, uttar pr...   

                       neighborhood                         street  city  \
4                                 0                              0  agra   
1756                       52gv+ch2                       52gv+ch2  agra   
516                               0                              0  agra   
1056             great gate, 282001             great gate, 282001  agra   
15                                0                              0  agra   
1703  sikandra - bhagwan talkies rd  sikandra - bhagwan talkies rd  agra   
2913                       52hf+69m                       52hf+69m  agra   

      postalCode          state  location_lat  ...  categories_2 categories_3  \
4         282001  uttar pradesh     27.175145  ...      monument            0   
1756      282006  uttar pradesh     27.176015  ...             0            0   
516       282007  uttar pradesh     27.220556  ...             0            0   
1056      282001  uttar pradesh     27.171714  ...             0            0   
15        282003  uttar pradesh     27.179533  ...      fortress            0   
1703      282007  uttar pradesh     27.216529  ...             0            0   
2913      282003  uttar pradesh     27.178081  ...             0            0   

     categories_4 categories_5 categories_6  \
4               0            0            0   
1756            0            0            0   
516             0            0            0   
1056            0            0            0   
15              0            0            0   
1703            0            0            0   
2913            0            0            0   

                                         reviews_0_text  \
4     this should be six star, one extra for the fee...   
1756  great place to sit and relax, especially for s...   
516   a very calm and historical place , entry ticke...   
1056  the gate to heaven. you cannot possibly antici...   
15    a historical fort having incredible architectu...   
1703  named as kanch mahal, but there is no kanch......   
2913                                                 ❤️   

                                         reviews_1_text  \
4     no doubt it's 7 out of wonders, great history ...   
1756  agra development authority has developed a per...   
516   not much to see other than the khabar of akbar...   
1056  darwaza-i-rauza is one of the components of th...   
15    great place for the ones who love history. sur...   
1703  kanch mahal is located beside the akbar tomb &...   
2913  palace is situated between the white marble kh...   

              SubcategoryName final_score  \
4                    monument    9.489384   
1756       tourist attraction    4.585088   
516       historical landmark    6.378584   
1056       tourist attraction    5.534232   
15                   fortress    8.863912   
1703       tourist attraction    4.654902   
2913  historical place museum    2.974391   

                                            feature_set  
4     careful monument few uttar pra